In [ ]:
"""

Portfolio Optimisation using Lagrange multipliers and Monte Carlo Simulation

This script simulates 10,000 random portfolios and identifies the portfolio with minimum variance.
It also uses Lagrange multipliers to minimise variance.
Variance is minimised to minimise risk.
The efficient frontier is plotted to visualise the risk-return trade-off of randomly generated portfolios.

"""

In [ ]:
#Installs financial data
%pip install yfinance
%pip install matplotlib

#Imports
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def calculate_covariance_matrix(daily_percentage_returns):
    """
    Calculate the covariance matrix from daily percentage returns.
    
    Args:
        daily_percentage_returns (pd.DataFrame): DataFrame containing daily returns
        
    Returns:
        np.ndarray: Covariance matrix of the returns
    """

    return daily_percentage_returns.cov().values

def calculate_portfolio_variance(w, cov_matrix):
    """
    Calculate the portfolio variance given asset weights and the covariance matrix.
    
    Uses the formula: σ² = w^T * Σ * w
    where w is the weight vector and Σ is the covariance matrix
    
    Args:
        weights (np.ndarray): Portfolio weights for each asset
        cov_matrix (np.ndarray): Covariance matrix of asset returns
        
    Returns:
        float: Portfolio variance
    """

    return np.dot(w.T, np.dot(cov_matrix, w))

def calculate_portfolio_volatility(weights, cov_matrix):
    """
    Calculate the standard deviation (volatility) of a portfolio.
    
    Args:
        weights (np.ndarray): Portfolio weights for each asset
        cov_matrix (np.ndarray): Covariance matrix of asset returns
        
    Returns:
        float: Portfolio volatility (standard deviation)
    """

    return np.sqrt(calculate_portfolio_variance(weights, cov_matrix))

def calculate_expected_returns(weights, returns):
    """
    Calculate the expected returns of a portfolio.
    
    Args:
        weights (np.ndarray): Portfolio weights for each asset
        returns (pd.DataFrame): Historical returns data
        
    Returns:
        float: Expected portfolio return
    """

    expected_returns = np.dot(weights, returns.mean().values)
    return expected_returns

def generate_random_weights():
    """
    Generate four random numbers that sum to 1 using Dirichlet distribution.
    The Dirichlet distribution ensures all weights are positive and sum to 1.
    
    Returns:
        np.ndarray: Array of 4 random weights that sum to 1
    """

    numbers = np.random.dirichlet([1, 1, 1, 1])
    return numbers

In [ ]:
def optimise_portfolio_lagrange(ones, cov_matrix):
  """
    Calculate the minimum variance portfolio using Lagrange multipliers method.
    
    This function implements the formula derived from Lagrange:
    w* = (Σ^-1 * 1) / (1^T * Σ^-1 * 1)
    where Σ is the covariance matrix and 1 is a vector of ones
    
    Args:
        ones_vector (np.ndarray): Vector of ones (constraint vector)
        cov_matrix (np.ndarray): Covariance matrix of asset returns
        
    Returns:
        np.ndarray: Optimal portfolio weights
    """
  
  inv_cov = np.linalg.inv(cov_matrix)
  numerator = np.dot(inv_cov, ones)
  denominator = np.dot(ones.T, np.dot(inv_cov, ones))
  return numerator / denominator

def get_optimal_portfolio_lagrange(companies, returns):
    """
    Find the optimal minimum variance portfolio using Lagrange multipliers.
    
    This function coordinates the optimization process and calculates
    portfolio metrics for the optimal weights.
    
    Args:
        companies (list): List of company ticker symbols
        returns (pd.DataFrame): Historical returns data
        
    Returns:
        tuple: (optimal_weights, variance, expected_returns)
            - optimal_weights (np.ndarray): Optimal portfolio weights (as decimals)
            - variance (float): Portfolio variance
            - expected_returns (float): Expected portfolio returns
    """

    ones = np.ones(len(companies)) #An array of ones (needed for the formula)
    cov_matrix = calculate_covariance_matrix(returns) #covariance matrix
    optimal_weights = optimise_portfolio_lagrange(ones, cov_matrix) #optimal portfolio
    variance = calculate_portfolio_variance(optimal_weights, cov_matrix) #Calculate variance of the optimal portfolio
    expected_returns = calculate_expected_returns(optimal_weights, returns) #Calculate expected returns from this portfolio
    
    return optimal_weights, variance, expected_returns

In [ ]:
def optimise_portfolio_monte_carlo(returns):
    """
    Find the minimum variance portfolio using Monte Carlo simulation.
    
    This function generates 10,000 random portfolios and selects the one
    with minimum variance (volatility).
    
    Args:
        returns (pd.DataFrame): Historical returns data
        
    Returns:
        tuple: (best_weights, min_volatility, all_volatilities, all_returns, min_index)
            - best_weights (np.ndarray): Weights of minimum variance portfolio
            - min_volatility (float): Minimum volatility achieved
            - all_volatilities (list): Volatilities of all simulated portfolios
            - all_returns (list): Returns of all simulated portfolios  
            - min_index (int): Index of minimum variance portfolio
    """

    cov_matrix = calculate_covariance_matrix(returns)
    
    #Storage for simulation results
    all_volatilities = []
    all_returns = []
    all_weights = []
    
    #Creates 10000 randomly weighted portfolios and adds it to the lists
    for i in range(10000):
        random_portfolio = generate_random_weights()
        all_weights.append(random_portfolio)
        all_volatilities.append(calculate_portfolio_volatility(random_portfolio, cov_matrix))
        all_returns.append(calculate_expected_returns(random_portfolio, returns))

        min_var_index = np.argmin(all_volatilities)
    return all_weights[min_var_index], all_volatilities[min_var_index], all_volatilities, all_returns, min_var_index

def get_optimal_portfolio_monte_carlo(returns):
    """
    Function to get optimal portfolio results from Monte Carlo simulation.
    
    Args:
        returns (pd.DataFrame): Historical returns data
        
    Returns:
        tuple: (optimal_weights, optimal_volatility, all_volatilities, all_returns, min_index)
    """

    optimal_portfolio = optimise_portfolio_monte_carlo(returns)
    optimal_weights_monte_carlo = optimal_portfolio[0]
    optimal_volatility_monte_carlo = optimal_portfolio[1]
    all_volatilities = optimal_portfolio[2]
    all_returns = optimal_portfolio[3]
    min_var_index = optimal_portfolio[4]
    return optimal_weights_monte_carlo, optimal_volatility_monte_carlo, all_volatilities, all_returns, min_var_index

In [ ]:
def plot_efficient_frontier(volatilities, returns, min_variance_index):
    """
    Plots the Efficient Frontier showing risk-return trade-off.
    
    The efficient frontier shows the set of optimal portfolios offering
    the highest expected return for each level of risk.
    
    Args:
        volatilities (list): List of portfolio volatilities from simulation
        returns (list): List of portfolio returns from simulation
        min_variance_index (int): Index of the minimum variance portfolio
    """

    plt.figure(figsize=(10,6))

    #Plot all simulated portfolios as blue dots
    plt.scatter(volatilities, returns, c='blue', s=5, alpha=0.3)
    #Highlight minimum variance portfolio with green star
    plt.scatter(volatilities[min_variance_index], returns[min_variance_index], c='green', marker = '*', s=200, label = 'Minimum Variance Portfolio')
    plt.legend()
    #Add labels and formatting
    plt.xlabel('Volatility (Standard Deviation)')
    plt.ylabel('Expected Return')
    plt.title('Efficient Frontier (Random Portfolios)')
    plt.show()

In [ ]:
#Main execution code. Run both optimisation methods every year from 2015-2025

#Defines the companies to be analysed and time windows
companies = ['TSLA', 'NVDA', 'GOOGL', 'AMZN']
dates = ["2015-01-01", "2016-01-01", "2017-01-01", "2018-01-01", "2019-01-01", "2020-01-01", "2021-01-01", "2022-01-01", "2023-01-01", "2024-01-01", "2025-01-01"]

print("Optimised portfolio using Lagrange and Monte Carlo methods to minimise variance\n\n\n")

#Loop through each 1-year period
for i in range(len(dates) - 1):
    start_date = dates[i]
    end_date = dates[i + 1]
    
    #Download historical closing price data for the selected time period
    data = yf.download(companies, start_date, end_date)['Close']

    #Calculate daily returns and drop missing values
    data_returns = data.pct_change().dropna()

    #Compute the covariance matrix of returns
    cov_matrix = calculate_covariance_matrix(data_returns)


    # ----- LAGRANGE MULTIPLIERS METHOD -----
    results_lagrange = get_optimal_portfolio_lagrange(companies, data_returns)
    optimal_weights_using_lagrange = results_lagrange[0]
    variance_lagrange = results_lagrange[1]
    expected_returns_lagrange = results_lagrange[2]

    print(f"Lagrange between {start_date} and {end_date}:")
    for j in range(4):
        #Print weights as percentages
        print(companies[j] + ":", optimal_weights_using_lagrange[j] * 100)
    print("Expected returns: ", expected_returns_lagrange * 100)
    print("Variance:", variance_lagrange)
    print("\n")

    # -----MONTE CARLO SIMULATION METHOD -----
    results_monte_carlo = get_optimal_portfolio_monte_carlo(data_returns)
    optimal_weights_monte_carlo = results_monte_carlo[0]
    optimal_volatility_monte_carlo = results_monte_carlo[1]
    all_volatilities = results_monte_carlo[2]
    all_returns = results_monte_carlo[3]
    min_var_index = results_monte_carlo[4]

    print(f"Monte Carlo Simulations between {start_date} and {end_date}:")
    for k in range(len(companies)):
        print(f"{companies[k]}:", optimal_weights_monte_carlo[k] * 100)
    #Calculate and print expected return from optimal weights
    print("Expected returns:", calculate_expected_returns(optimal_weights_monte_carlo, data_returns) * 100)
    print("Variance:", optimal_volatility_monte_carlo * optimal_volatility_monte_carlo)
    print("\n")

    #Plot the efficient frontier for the chosen year (Monte Carlo method)
    plot_efficient_frontier(all_volatilities, all_returns, min_var_index)
    print("\n\n\n\n\n")
print("Suggested portfolio weights to minimise variance (average of all years from 2015 to 2025): ")
# ----- LAGRANGE MULTIPLIERS METHOD -----
final_start_date = dates[0]
final_end_date = dates[-1]
#Download historical closing price data for the selected time period
final_data = yf.download(companies, final_start_date, final_end_date)['Close']
final_data_returns = final_data.pct_change().dropna()

final_results_lagrange = get_optimal_portfolio_lagrange(companies, final_data_returns)
final_weights_lagrange = final_results_lagrange[0]
final_variance_lagrange = final_results_lagrange[1]
final_expected_returns_lagrange = final_results_lagrange[2]
for j in range(4):
    #Print weights as percentages
    print(companies[j] + ":", final_weights_lagrange[j] * 100)
print("Expected returns: ", final_expected_returns_lagrange * 100)
print("Variance:", final_variance_lagrange)
print("\n")
# -----MONTE CARLO SIMULATION METHOD -----
final_results_monte_carlo = get_optimal_portfolio_monte_carlo(final_data_returns)
final_weights_monte_carlo = final_results_monte_carlo[0]
final_volatility_monte_carlo = final_results_monte_carlo[1]
final_all_volatilities = final_results_monte_carlo[2]
final_all_returns = final_results_monte_carlo[3]
final_min_var_index = final_results_monte_carlo[4]
for k in range(len(companies)):
    print(f"{companies[k]}:", final_weights_monte_carlo[k] * 100)
#Calculate and print expected return from optimal weights
print("Expected returns:", calculate_expected_returns(final_weights_monte_carlo, final_data_returns) * 100)
print("Variance:", final_volatility_monte_carlo * final_volatility_monte_carlo)
print("\n")
#Plot the efficient frontier for the chosen year (Monte Carlo method)
plot_efficient_frontier(final_all_volatilities, final_all_returns, final_min_var_index)
print("\n\n\n\n\n")
